<a href="https://colab.research.google.com/github/alexisakov/NLPTutorial/blob/main/NLPExcercise01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!cp mystem /root/.local/bin/mystem

--2021-08-12 06:13:54--  http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving download.cdn.yandex.net (download.cdn.yandex.net)... 5.45.205.243, 5.45.205.245, 5.45.205.242, ...
Connecting to download.cdn.yandex.net (download.cdn.yandex.net)|5.45.205.243|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://cache-mskm901.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz [following]
--2021-08-12 06:13:55--  http://cache-mskm901.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving cache-mskm901.cdn.yandex.net (cache-mskm901.cdn.yandex.net)... 5.45.220.11, 2a02:6b8:0:2002::12
Connecting to cache-mskm901.cdn.yandex.net (cache-mskm901.cdn.yandex.net)|5.45.220.11|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16457938 (16M) [application/octet-stream]
Saving to: ‘mystem-3.0-linux3.1-64bit.tar.gz.1’

mystem-3.0-linux3.1 100%[===================>] 

In [12]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

In [40]:
from pymystem3 import Mystem
tqdm.pandas() 

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [6]:
df_train = pd.read_csv('https://github.com/alexisakov/NLPTutorial/raw/main/train.csv.zip'
  , sep=';')
df_test = pd.read_csv('https://github.com/alexisakov/NLPTutorial/raw/main/test.csv.zip'
  , sep=';')

In [7]:
df_train['text'] = df_train['text'].progress_map(lambda x: BeautifulSoup(x, 'html.parser').text)

In [8]:
df_train.head()

,id,text,target
0,0,В молодой и дружный коллектив динамично развив...,0
1,1,Обязанности: - Прием входящих звонков и заявок...,0
2,2,Федеральная сеть аптек в связи с активным расш...,0
3,3,Обязанности: разработка проектной и рабочей ...,0
4,4,Производителю сантехники тм Domani Spa требуют...,1


In [41]:
mystm = Mystem()

In [14]:
df_train['text'] = df_train['text'].progress_map(lambda x: ''.join(mystm.lemmatize(x)).rstrip() )

In [15]:
df_train.head()

,id,text,target
0,0,в молодой и дружный коллектив динамично развив...,0
1,1,обязанность: - прием входить звонок и заявка;-...,0
2,2,федеральный сеть аптека в связь с активный рас...,0
3,3,обязанность: разработка проектный и рабочий ...,0
4,4,производитель сантехника тм Domani Spa требова...,1


In [16]:
tfidf_vectorizer=TfidfVectorizer(use_idf=True, max_df=0.95)

In [17]:
tfidf_vectorizer.fit(df_train['text'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=0.95, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [18]:
X = tfidf_vectorizer.transform(df_train['text'])

In [19]:
Y = df_train['target']

In [20]:
log_reg = LogisticRegression(verbose=1, solver='liblinear',random_state=0)
rand_for = RandomForestClassifier(random_state=0)

In [21]:
parameters = {
     'C': [0.1,0.5,1.0,2.0,5.0], 'penalty': ['l2','l1']}
gs_lr = GridSearchCV(log_reg, parameters,
                   scoring = 'roc_auc',
                   cv = 5)

In [22]:
gs_lr.fit(X,Y)

[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]

GridSearchCV(cv=5, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=0, solver='liblinear',
                                          tol=0.0001, verbose=1,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [0.1, 0.5, 1.0, 2.0, 5.0],
                         'penalty': ['l2', 'l1']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='roc_auc', verbose=0)

In [24]:
gs_lr.best_params_

{'C': 5.0, 'penalty': 'l2'}

In [34]:
parameters = {
     'max_depth': [20,50]}
gs_rf = GridSearchCV(rand_for, parameters,
                   scoring = 'roc_auc',
                   cv = 5)

In [35]:
gs_rf.fit(X,Y)

GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False, random_state=0,
                                   

In [36]:
gs_rf.best_params_

{'max_depth': 50}

# Apply the model to test data

In [37]:
df_test['text'] = df_test['text'].progress_map(lambda x: BeautifulSoup(x, 'html.parser').text)

In [42]:
df_test['text'] = df_test['text'].progress_map(lambda x: ''.join(mystm.lemmatize(x)).rstrip() )

In [43]:
X_test = tfidf_vectorizer.transform(df_test['text'])

In [44]:
df_test.head()

,id,text
0,31063,в крупный компания по организация и приготовле...
1,31064,обязанность: обеспечение необходимый функцион...
2,31065,обязанность: отгрузка и прием товар со скла...
3,31066,обязанность: приготовление холодный и горячий...
4,31067,вакансия срочный! внимание! просьба подробно и...


In [ ]:
df_test[['target']] = gs_lr.best_estimator_.predict_proba(X_test)[:,1]

In [45]:
df_test[['target']] = gs_rf.best_estimator_.predict_proba(X_test)[:,1]

In [46]:
df_test.head()

,id,text,target
0,31063,в крупный компания по организация и приготовле...,0.869119
1,31064,обязанность: обеспечение необходимый функцион...,0.163827
2,31065,обязанность: отгрузка и прием товар со скла...,0.866258
3,31066,обязанность: приготовление холодный и горячий...,0.817055
4,31067,вакансия срочный! внимание! просьба подробно и...,0.494838


In [ ]:
df_test.head()

,id,text,target
0,31063,в крупный компания по организация и приготовле...,0.999758
1,31064,обязанность: обеспечение необходимый функцион...,0.004606
2,31065,обязанность: отгрузка и прием товар со скла...,0.998496
3,31066,обязанность: приготовление холодный и горячий...,0.990171
4,31067,вакансия срочный! внимание! просьба подробно и...,0.923102


In [47]:
from google.colab import files

In [ ]:
df_test[{'id','target'}].head()

,id,target
0,31063,0.999758
1,31064,0.004606
2,31065,0.998496
3,31066,0.990171
4,31067,0.923102


In [48]:
df_test[{'id','target'}].to_csv('scoredtest.csv',index=False) 
files.download('scoredtest.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>